In [26]:
#Importing libraries
from tweepy import OAuthHandler
from tweepy import Cursor
from tweepy import API
import pandas as pd
import re
from textblob import TextBlob
import plotly.graph_objects as go

In [27]:
# Getting twitter Authentication credentials
CONSUMER_KEY="jJyU52WnIin7QcfcwBtBBcQ2m"
CONSUMER_SECRET="z0Bv43nI1LYef4fvmL8hSO0ywYxryMuF5fnYZGvK2KCpDOJXx4"
ACCESS_TOKEN="1382621002832572419-ETl0oJcJxJp3bsDSNwllnfIFjbqk7o"
ACCESS_TOKEN_SECRET="9EaW2JHW1JKEz1e4ig4lblxVXeZ6oXVnetJYNMg0GqT8a"
auth=OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

# Create API object
api=API(auth,wait_on_rate_limit=True)

# Extracting tweets from twitter

no_tweets=30

bjp_posts1=Cursor(api.search, q='#BJP', lang="en", since="2020-01-01").items(no_tweets)
bjp_posts2=Cursor(api.search, q='#NDA', lang="en", since="2020-01-01").items(no_tweets)
bjp_posts3=Cursor(api.search, q='#Narendra Modi', lang="en", since="2020-01-01").items(no_tweets)


inc_posts1=Cursor(api.search, q='#INC', lang="en", since="2020-01-01").items(no_tweets)
inc_posts2=Cursor(api.search, q='#Congress', lang="en", since="2020-01-01").items(no_tweets)
inc_posts3=Cursor(api.search, q='#Rahul Gandhi', lang="en", since="2020-01-01").items(no_tweets)

In [28]:
#Creating a dataframe with a column called Tweets

bjp_df=pd.DataFrame([tweet.text for tweet in bjp_posts1],columns=['Tweets'])
bjp_df1=pd.DataFrame([tweet.text for tweet in bjp_posts2],columns=['Tweets'])
bjp_df2=pd.DataFrame([tweet.text for tweet in bjp_posts3],columns=['Tweets'])
bjp_df = pd.concat([bjp_df,bjp_df1,bjp_df2], ignore_index=True)

inc_df=pd.DataFrame([tweet.text for tweet in inc_posts1],columns=['Tweets'])
inc_df1=pd.DataFrame([tweet.text for tweet in inc_posts2],columns=['Tweets'])
inc_df2=pd.DataFrame([tweet.text for tweet in inc_posts3],columns=['Tweets'])
inc_df = pd.concat([inc_df,inc_df1,inc_df2], ignore_index=True)

#For Demonstration
bjp_df

,Tweets
0,RT @Dev_Fadnavis: Attended @BJP4Maharashtra le...
1,Reports of theaters filled with chants of “Bha...
2,RT @DrAMSinghvi: Non #BJP opposition is the ne...
3,RT @lindanewmai: BJP only party that respects ...
4,RT @vramalin: What a shame #BJP they have mess...
...,...
65,"RT @telugustop: After praising PM Modi, NCP le..."
66,"After praising PM Modi, NCP leader takes a u-t..."
67,RT @telugustop: Modi meets leading Sikh intell...
68,Modi meets leading Sikh intellectual voices ht...


In [29]:
#Removing unusual words and symbols from the Tweets
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9_s:s]+','',text)     # Removed @...
    text=re.sub(r'#[A-Za-z0-9_s:s]+','',text)     # Removed #...
    text=re.sub(r'hmm ','',text)                  # Removed huu
    text=re.sub(r'[Hh]i ','',text)                # Removed Hi
    text=re.sub(r'and ','',text)                  # Removed and
    text=re.sub(r'oh ','',text)                   # Removed oh
    text=re.sub(r'via ','',text)                  # Removed via
    text=re.sub(r'RT[\s]+','',text)               # Removed RT
    text=re.sub(r'https[:s]\/\/\S+','',text)      # Removed https link
    return text

bjp_df['Tweets']=bjp_df['Tweets'].apply(cleanTxt)

inc_df['Tweets']=inc_df['Tweets'].apply(cleanTxt)

#For Demonstration
bjp_df

,Tweets
0,Attended leaders’ meeting in Mumbai this afte...
1,Reports of theaters filled with chants of “Bha...
2,Non opposition is the need of the hour. is a...
3,BJP only party that respects all former PMs: P...
4,What a shame they have messed up the politics...
...,...
65,"After praising PM Modi, NCP leader takes a u-t..."
66,"After praising PM Modi, NCP leader takes a u-t..."
67,Modi meets leading Sikh intellectual voices
68,Modi meets leading Sikh intellectual voices


In [30]:
# Creat a fun. to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Creat a new columns

bjp_df['Polarity']=bjp_df['Tweets'].apply(getPolarity)

inc_df['Polarity']=inc_df['Tweets'].apply(getPolarity)

#For Demonstration
bjp_df

,Tweets,Polarity
0,Attended leaders’ meeting in Mumbai this afte...,0.000000
1,Reports of theaters filled with chants of “Bha...,0.400000
2,Non opposition is the need of the hour. is a...,0.000000
3,BJP only party that respects all former PMs: P...,0.000000
4,What a shame they have messed up the politics...,0.250000
...,...,...
65,"After praising PM Modi, NCP leader takes a u-t...",0.000000
66,"After praising PM Modi, NCP leader takes a u-t...",0.000000
67,Modi meets leading Sikh intellectual voices,0.300000
68,Modi meets leading Sikh intellectual voices,0.300000


In [31]:
# Creat a function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

# Creat a new columns
    
bjp_df['Analysis']=bjp_df['Polarity'].apply(getAnalysis)

inc_df['Analysis']=inc_df['Polarity'].apply(getAnalysis)

# Show the dataframe
bjp_df

,Tweets,Polarity,Analysis
0,Attended leaders’ meeting in Mumbai this afte...,0.000000,Neutral
1,Reports of theaters filled with chants of “Bha...,0.400000,Positive
2,Non opposition is the need of the hour. is a...,0.000000,Neutral
3,BJP only party that respects all former PMs: P...,0.000000,Neutral
4,What a shame they have messed up the politics...,0.250000,Positive
...,...,...,...
65,"After praising PM Modi, NCP leader takes a u-t...",0.000000,Neutral
66,"After praising PM Modi, NCP leader takes a u-t...",0.000000,Neutral
67,Modi meets leading Sikh intellectual voices,0.300000,Positive
68,Modi meets leading Sikh intellectual voices,0.300000,Positive


In [32]:
# Show the dataframe
inc_df

,Tweets,Polarity,Analysis
0,Hardly any post-election analyses on social me...,-0.129167,Negative
1,"""The true measure of any society can be found ...",0.116667,Positive
2,"Our country's identity is not new, from the an...",-0.068182,Negative
3,"""The true measure of any society can be found ...",0.116667,Positive
4,"Our country's identity is not new, from the an...",-0.068182,Negative
...,...,...,...
82,My Leader Radhu Gandhi\n \nMission \n …,0.000000,Neutral
83,My Leader Radhu Gandhi\n \nMission \n …,0.000000,Neutral
84,My Leader Radhu Gandhi\n \nMission \n …,0.000000,Neutral
85,Gand.please pilot laoo congress bachaooo.,0.000000,Neutral


In [33]:
# Removing all nutral tweets

bjp_df.drop(bjp_df[bjp_df['Analysis']=='Neutral'].index, inplace=True)

inc_df.drop(inc_df[inc_df['Analysis']=='Neutral'].index, inplace=True)

# Calculating Percentage of positive tweets

a=bjp_df.groupby('Analysis').count();
b=inc_df.groupby('Analysis').count();
print("BJP:\n",a)
print("\nINC:\n",b)

bjp_pos=round(a['Tweets'][1]*100/bjp_df.shape[0])
inc_pos=round(b['Tweets'][1]*100/inc_df.shape[0])

BJP:
           Tweets  Polarity
Analysis                  
Negative       8         8
Positive      34        34

INC:
           Tweets  Polarity
Analysis                  
Negative      13        13
Positive      26        26


In [34]:
# Plotting graph

Party=['B J P','I N C']
pos=[ bjp_pos, inc_pos ]
neg=[ 100-bjp_pos, 100-inc_pos ]

fig=go.Figure(data=[
    go.Bar(name='+ve',x=Party,y=pos),
    go.Bar(name='-ve',x=Party,y=neg)
])

fig.update_layout(title_text="Analysis of Tweets For Both Party",
                  title_font_size=20,
                  barmode='group')
fig.show()

In [35]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# feature extraction
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, stop_words='english')

#For BJP..................>
# training and testing
bjp_bow = bow_vectorizer.fit_transform(bjp_df['Tweets'])
x_train, x_test, y_train, y_test = train_test_split(bjp_bow, bjp_df['Analysis'], random_state=42, test_size=0.25)

model = LogisticRegression()
model.fit(x_train, y_train)

pred_train = model.predict(x_train)
bjp_acu_train=accuracy_score(y_train,pred_train)

pred_test = model.predict(x_test)
bjp_acu_test=accuracy_score(y_test,pred_test)

print("Training Accuracy:",bjp_acu_train)
print("Testing Accuracy:",bjp_acu_test)

Training Accuracy: 0.967741935483871
Testing Accuracy: 1.0


In [36]:
#For INC..................>
# training and testing
inc_bow = bow_vectorizer.fit_transform(inc_df['Tweets'])
x_train, x_test, y_train, y_test = train_test_split(inc_bow, inc_df['Analysis'], random_state=42, test_size=0.25)

model = LogisticRegression()
model.fit(x_train, y_train)

pred_train = model.predict(x_train)
inc_acu_train=accuracy_score(y_train,pred_train)

pred_test = model.predict(x_test)
inc_acu_test=accuracy_score(y_test,pred_test)

print("Training Accuracy:",inc_acu_train)
print("Testing Accuracy:",inc_acu_test)

Training Accuracy: 0.9655172413793104
Testing Accuracy: 1.0
